The objecective of this notebook is to
1. Read the volume agg data for exitsing stores as per history
2. Read the dummy volume data for new stores as per submission
3. use only the store combinations given in submission

In [1]:
# import jupyter_black

# jupyter_black.load()

In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    r"/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/abi_hackathon_2k24


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)
from pandarallel import pandarallel
import re
from datetime import datetime
import gc
from sklearn.preprocessing import FunctionTransformer

# from helpers.compress import reduce_mem_usage
import pickle
import warnings

warnings.filterwarnings("ignore")

In [7]:
pandarallel.initialize(progress_bar=True, nb_workers=os.cpu_count() - 2)

INFO: Pandarallel will run on 14 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
data_folder = "data"
macro_data_folder = "macro_data"
processed_data_folder = "processed_data_S2"
processed_macro = "macro_processed"
fe_data = "feature_engineered_data"

In [ ]:
volume_monthly_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_data_folder, "volume_df_month_agg.pkl")
)

id_life_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_data_folder, "id_date_life.pkl")
)

In [ ]:
sample_submission_df = pd.read_csv(
    os.path.join(os.getcwd(), data_folder, "sample_submission_file.csv")
)

In [ ]:
volume_monthly_df.head()

In [ ]:
volume_monthly_df.drop_duplicates(['poc_ppg_id','year','month']).shape

In [ ]:
volume_monthly_df.shape

In [ ]:
# adding synthetic data for the new ones

In [12]:
synthetic_ids_monthly_df = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "synthetic_poc_ppg_monthly_vols_v2.pkl"
    )
)

poc_ppg_combination_clusters_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_data_folder, "poc_ppg_combination_clusters.pkl")
)

In [13]:
calendar_df_month = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "calendar_df_month.pkl")
)

In [14]:
synthetic_ids_monthly_df.shape

(280992, 17)

In [15]:
synthetic_ids_monthly_df.drop_duplicates(['poc_ppg_id','year','month']).shape

(280992, 17)

In [16]:
synthetic_ids_monthly_df["sales_volume_hl_agg"] = synthetic_ids_monthly_df[
    "sales_volume_hl_agg"
].fillna(0.0)

In [17]:
new_ids = synthetic_ids_monthly_df["poc_ppg_id"].unique()
len(new_ids)

5854

In [18]:
existing_ids = volume_monthly_df['poc_ppg_id'].unique()
intersect_ids = list(set(new_ids).intersection(set(existing_ids)))

In [19]:
synthetic_ids_monthly_df = synthetic_ids_monthly_df.loc[~synthetic_ids_monthly_df['poc_ppg_id'].isin(intersect_ids)].reset_index(drop=True)

In [20]:
synthetic_ids_monthly_df.shape

(280896, 17)

In [21]:
# adding data for 2023 only for new ids

In [22]:
new_id_df = synthetic_ids_monthly_df.drop_duplicates(
    ["unique_poc_id", "ppg_name", "poc_ppg_id"]
)[["unique_poc_id", "ppg_name", "poc_ppg_id"]].reset_index(drop=True)
new_id_df["key"] = 1

In [23]:
new_id_df.shape

(5852, 4)

In [24]:
calendar_df_month = calendar_df_month.loc[
    (calendar_df_month["year_month_start_date"] >= "2023-01-01")
    & (calendar_df_month["year_month_start_date"] <= "2023-12-01"),
    ["year", "month", "year_month_start_date"],
].reset_index(drop=True)
calendar_df_month["key"] = 1

In [25]:
calendar_df_month.shape

(12, 4)

In [26]:
new_id_df_23 = pd.merge(new_id_df, calendar_df_month, on="key").drop(["key"], axis=1)

In [27]:
new_id_df_23["sales_volume_hl_agg"] = 0.0

In [28]:
synthetic_ids_monthly_df = synthetic_ids_monthly_df.rename(
    columns={"date": "year_month_start_date"}
)

In [29]:
synthetic_ids_monthly_df = pd.concat(
    [synthetic_ids_monthly_df[volume_monthly_df.columns.tolist()], new_id_df_23],
    axis=0,
    ignore_index=True,
)

In [30]:
synthetic_ids_monthly_df["year_month_start_date"].max()

Timestamp('2023-12-01 00:00:00')

In [31]:
# Join with monthly vol from history and new id history created

In [32]:
volume_monthly_df = pd.concat(
    [volume_monthly_df, synthetic_ids_monthly_df[volume_monthly_df.columns.tolist()]],
    axis=0,
    ignore_index=True,
)

In [33]:
volume_monthly_df = volume_monthly_df.sort_values(
    ["poc_ppg_id", "year", "month"]
).reset_index(drop=True)

In [34]:
volume_monthly_df.shape

(79400788, 7)

In [35]:
volume_monthly_df.drop_duplicates(['poc_ppg_id','year','month']).shape

(79400788, 7)

In [36]:
# adding cluster details

In [37]:
poc_ppg_combination_clusters_df.shape

(8318715, 7)

In [38]:
poc_ppg_combination_clusters_df.drop_duplicates(['poc_ppg']).shape

(8318715, 7)

In [39]:
poc_ppg_combination_clusters_df = poc_ppg_combination_clusters_df.rename(
    columns={"poc_ppg": "poc_ppg_id"}
)

In [40]:
poc_ppg_combination_clusters_df["poc_ppg_cluster"] = (
    poc_ppg_combination_clusters_df["poc_cluster"].astype(str)
    + "_"
    + poc_ppg_combination_clusters_df["ppg_cluster"].astype(str)
)

In [41]:
volume_monthly_df.head()

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date
0,AAAA__AK_AB_CG_T,AK AB CG T,AAAA,2019,4,0.065803,2019-04-01
1,AAAA__AK_AB_CG_T,AK AB CG T,AAAA,2019,5,0.131605,2019-05-01
2,AAAA__AK_AB_CG_T,AK AB CG T,AAAA,2019,6,0.000000,2019-06-01
3,AAAA__AK_AB_CG_T,AK AB CG T,AAAA,2019,7,0.000000,2019-07-01
4,AAAA__AK_AB_CG_T,AK AB CG T,AAAA,2019,8,0.000000,2019-08-01


In [42]:
volume_monthly_df = pd.merge(
    volume_monthly_df,
    poc_ppg_combination_clusters_df[
        ["poc_ppg_id", "sample_submission_flag", "id_life_flag", "poc_ppg_cluster"]
    ],
    on="poc_ppg_id",
    how="left",
)

In [43]:
volume_monthly_df.shape

(79400788, 10)

In [44]:
volume_monthly_df.drop_duplicates(['poc_ppg_id','year','month']).shape

(79400788, 10)

In [45]:
volume_monthly_df.isnull().sum()  # misiing one are delisted so not clustered

poc_ppg_id                      0
ppg_name                        0
unique_poc_id                   0
year                            0
month                           0
sales_volume_hl_agg             0
year_month_start_date           0
sample_submission_flag    6898759
id_life_flag              6898759
poc_ppg_cluster           6898759
dtype: int64

In [46]:
volume_monthly_df.to_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "all_id_vol_cluster_monthly_df.pkl"
    )
)

In [47]:
# volume_monthly_df = pd.read_pickle(
#     os.path.join(
#         os.getcwd(), processed_data_folder, "all_id_vol_cluster_monthly_df.pkl"
#     )
# )

##### adding  metadata

In [48]:
poc_atrb_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "poc_atrb_df.pkl")
)
ppg_atrb_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "ppg_atrb_df.pkl")
)

In [49]:
print(ppg_atrb_df.shape)
print(ppg_atrb_df.drop_duplicates(['ppg_name']).shape)

(240, 7)
(240, 7)


In [50]:
print(poc_atrb_df.shape)
print(poc_atrb_df.drop_duplicates(['unique_poc_id']).shape)

(43645, 6)
(43645, 6)


In [51]:
# adding PPG details
volume_monthly_df = pd.merge(
    volume_monthly_df,
    ppg_atrb_df[["ppg_name", "category_name", "price_segment", "brand", "subbrand"]],
    on=["ppg_name"],
    how="left",
)

In [52]:
# adding poc details
volume_monthly_df = pd.merge(
    volume_monthly_df,
    poc_atrb_df[["unique_poc_id", "subchannel", "channel_type", "segment"]],
    on=["unique_poc_id"],
    how="left",
)

In [53]:
volume_monthly_df.isnull().sum()

poc_ppg_id                      0
ppg_name                        0
unique_poc_id                   0
year                            0
month                           0
sales_volume_hl_agg             0
year_month_start_date           0
sample_submission_flag    6898759
id_life_flag              6898759
poc_ppg_cluster           6898759
category_name                   0
price_segment                   0
brand                           0
subbrand                        0
subchannel                      0
channel_type                    0
segment                         0
dtype: int64

In [54]:
volume_monthly_df.shape

(79400788, 17)

###### seasonal index

In [55]:
volume_monthly_df["ppg_seasonality_index"] = volume_monthly_df.groupby(
    ["category_name", "price_segment", "month"]
)["sales_volume_hl_agg"].transform(lambda x: x.sum()) / volume_monthly_df.groupby(
    ["category_name", "price_segment"]
)[
    "sales_volume_hl_agg"
].transform(
    lambda x: x.sum()
)

In [56]:
volume_monthly_df["poc_seasonality_index"] = volume_monthly_df.groupby(
    ["segment", "month"]
)["sales_volume_hl_agg"].transform(lambda x: x.sum()) / volume_monthly_df.groupby(
    ["segment"]
)[
    "sales_volume_hl_agg"
].transform(
    lambda x: x.sum()
)

In [57]:
volume_monthly_df.isnull().sum()

poc_ppg_id                      0
ppg_name                        0
unique_poc_id                   0
year                            0
month                           0
sales_volume_hl_agg             0
year_month_start_date           0
sample_submission_flag    6898759
id_life_flag              6898759
poc_ppg_cluster           6898759
category_name                   0
price_segment                   0
brand                           0
subbrand                        0
subchannel                      0
channel_type                    0
segment                         0
ppg_seasonality_index           0
poc_seasonality_index           0
dtype: int64

In [58]:
volume_monthly_df.shape

(79400788, 19)

#### Todo
- Store category- brand  ranking
- Store Category -subrand ranking
- PPG Category - price segment - brand ranking

#### subsetting sample ids

In [59]:
# clean name
sample_submission_df["ppg_name_clean"] = sample_submission_df["ppg_name"].apply(
    lambda x: re.sub(r"\s+", "_", x)
)
# poc-Ppg id
sample_submission_df["poc_ppg_id"] = sample_submission_df.apply(
    lambda x: str(x["unique_poc_id"]) + "__" + str(x["ppg_name_clean"]), axis=1
)

In [60]:
best_ids = sample_submission_df["poc_ppg_id"].unique().tolist()
print(len(best_ids))

49337


In [61]:
set(best_ids).difference(set(volume_monthly_df["poc_ppg_id"].unique()))

set()

In [62]:
# volume for best id
volume_monthly_df_best = volume_monthly_df.loc[
    volume_monthly_df["poc_ppg_id"].isin(best_ids)
].reset_index(drop=True)

In [63]:
volume_monthly_df_best.isnull().sum()

poc_ppg_id                0
ppg_name                  0
unique_poc_id             0
year                      0
month                     0
sales_volume_hl_agg       0
year_month_start_date     0
sample_submission_flag    0
id_life_flag              0
poc_ppg_cluster           0
category_name             0
price_segment             0
brand                     0
subbrand                  0
subchannel                0
channel_type              0
segment                   0
ppg_seasonality_index     0
poc_seasonality_index     0
dtype: int64

In [64]:
volume_monthly_df_best.shape

(2565882, 19)

In [65]:
volume_monthly_df_best.drop_duplicates(['poc_ppg_id','year','month']).shape

(2565882, 19)

In [66]:
### Should we also include some dominant cluster  more combinations

##### only Lag features

In [67]:
def random_noise(dataframe):
    return np.random.normal(scale=0.01, size=(len(dataframe)))

In [68]:
def lag_features(dataframe, lags, id_col, target_col):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe["sales_lag_" + str(lag)] = dataframe.groupby([id_col])[
            target_col
        ].transform(
            lambda x: x.shift(lag)
            .fillna(method="bfill")
            .fillna(method="ffill")
            .fillna(0.0)
        ) + random_noise(
            dataframe
        )
    return dataframe

In [69]:
volume_monthly_df_best = lag_features(
    volume_monthly_df_best, [12, 13, 14, 15], "poc_ppg_id", "sales_volume_hl_agg"
)

In [70]:
volume_monthly_df_best.head()

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date,sample_submission_flag,id_life_flag,poc_ppg_cluster,category_name,price_segment,brand,subbrand,subchannel,channel_type,segment,ppg_seasonality_index,poc_seasonality_index,sales_lag_12,sales_lag_13,sales_lag_14,sales_lag_15
0,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,9,2.657029,2021-09-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.086632,0.083310,2.680361,2.659049,2.641487,2.663112
1,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,10,1.635095,2021-10-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.097920,0.099356,2.660838,2.665773,2.663828,2.645569
2,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,11,5.477568,2021-11-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.116346,0.114571,2.663475,2.656727,2.663527,2.641021
3,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,12,4.905285,2021-12-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.135510,0.138193,2.645500,2.674356,2.650064,2.654775
4,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2022,1,0.613161,2022-01-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.056200,0.055705,2.635591,2.657807,2.671511,2.658479


In [71]:
volume_monthly_df_best.shape

(2565882, 23)

##### lag diff

In [72]:
def calculate_lag_diff(dataframe):
    dataframe = dataframe.copy()
    diff_df = dataframe.filter(regex="sales_lag").diff(axis=1).dropna(axis=1, how="all")
    diff_df.columns = [col + "_diff" for col in diff_df.columns]
    dataframe = pd.concat([dataframe, diff_df], axis=1)
    return dataframe

In [73]:
volume_monthly_df_best = calculate_lag_diff(volume_monthly_df_best)

In [74]:
volume_monthly_df_best.isnull().sum()

poc_ppg_id                0
ppg_name                  0
unique_poc_id             0
year                      0
month                     0
sales_volume_hl_agg       0
year_month_start_date     0
sample_submission_flag    0
id_life_flag              0
poc_ppg_cluster           0
category_name             0
price_segment             0
brand                     0
subbrand                  0
subchannel                0
channel_type              0
segment                   0
ppg_seasonality_index     0
poc_seasonality_index     0
sales_lag_12              0
sales_lag_13              0
sales_lag_14              0
sales_lag_15              0
sales_lag_13_diff         0
sales_lag_14_diff         0
sales_lag_15_diff         0
dtype: int64

In [75]:
volume_monthly_df_best.shape

(2565882, 26)

#### roll features

In [76]:
def roll_mean_features(dataframe, windows, id_col, target_col):
    dataframe = dataframe.copy()
    for window in windows:
        dataframe["sales_roll_mean_" + str(window)] = dataframe.groupby([id_col])[
            target_col
        ].transform(
            lambda x: x.shift(12)
            .rolling(window=window, min_periods=1)
            .mean()
            .fillna(method="bfill")
            .fillna(method="ffill")
            .fillna(0.0)
        ) + random_noise(
            dataframe
        )
    return dataframe

In [77]:
volume_monthly_df_best = roll_mean_features(volume_monthly_df_best, [3, 6, 9], "poc_ppg_id", "sales_volume_hl_agg")

In [78]:
volume_monthly_df_best.isnull().sum()

poc_ppg_id                0
ppg_name                  0
unique_poc_id             0
year                      0
month                     0
sales_volume_hl_agg       0
year_month_start_date     0
sample_submission_flag    0
id_life_flag              0
poc_ppg_cluster           0
category_name             0
price_segment             0
brand                     0
subbrand                  0
subchannel                0
channel_type              0
segment                   0
ppg_seasonality_index     0
poc_seasonality_index     0
sales_lag_12              0
sales_lag_13              0
sales_lag_14              0
sales_lag_15              0
sales_lag_13_diff         0
sales_lag_14_diff         0
sales_lag_15_diff         0
sales_roll_mean_3         0
sales_roll_mean_6         0
sales_roll_mean_9         0
dtype: int64

In [79]:
volume_monthly_df_best.shape

(2565882, 29)

#### ewm features

In [80]:
def ewm_features(dataframe, alphas, lags, id_col, target_col):
    dataframe = dataframe.copy()
    for alpha in alphas:
        for lag in lags:
            dataframe[
                "sales_ewm_alpha_" + str(alpha).replace(".", "") + "_lag_" + str(lag)
            ] = dataframe.groupby([id_col])[target_col].transform(
                lambda x: x.shift(lag)
                .ewm(alpha=alpha, min_periods=1)
                .mean()
                .fillna(method="bfill")
                .fillna(method="ffill")
                .fillna(0.0)
            )
    return dataframe

In [81]:
volume_monthly_df_best = ewm_features(
    volume_monthly_df_best, [0.95, 0.9, 0.8], [12, 13, 14], "poc_ppg_id", "sales_volume_hl_agg"
)

In [82]:
volume_monthly_df_best.shape

(2565882, 38)

##### macro

In [83]:
ppg_financials = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "ppg_financials.pkl")
)

calendar_df_month = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "calendar_df_month.pkl")
)
covid_data_monthly = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "covid_data_monthly.pkl")
)
weather_data_monthly = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "weather_data_monthly.pkl")
)
macro_df = pd.read_pickle(
    os.path.join(os.getcwd(), processed_macro, "macro_eco_df_pvt.pkl")
)

In [84]:
volume_monthly_df_best = pd.merge(
    volume_monthly_df_best, ppg_financials, on=["ppg_name", "year", "month"], how="left"
)

# filling the missing values
volume_monthly_df_best["maco_hl"] = volume_monthly_df_best.groupby(["ppg_name", "month"])["maco_hl"].transform(
    lambda x: x.median()
)
volume_monthly_df_best["nr_hl"] = volume_monthly_df_best.groupby(["ppg_name", "month"])["nr_hl"].transform(
    lambda x: x.median()
)

volume_monthly_df_best.isnull().sum()

poc_ppg_id                    0
ppg_name                      0
unique_poc_id                 0
year                          0
month                         0
sales_volume_hl_agg           0
year_month_start_date         0
sample_submission_flag        0
id_life_flag                  0
poc_ppg_cluster               0
category_name                 0
price_segment                 0
brand                         0
subbrand                      0
subchannel                    0
channel_type                  0
segment                       0
ppg_seasonality_index         0
poc_seasonality_index         0
sales_lag_12                  0
sales_lag_13                  0
sales_lag_14                  0
sales_lag_15                  0
sales_lag_13_diff             0
sales_lag_14_diff             0
sales_lag_15_diff             0
sales_roll_mean_3             0
sales_roll_mean_6             0
sales_roll_mean_9             0
sales_ewm_alpha_095_lag_12    0
sales_ewm_alpha_095_lag_13    0
sales_ew

In [85]:
volume_monthly_df_best.shape

(2565882, 40)

In [86]:
calendar_df_month.head()

volume_monthly_df_best = pd.merge(
    volume_monthly_df_best,
    calendar_df_month[
        [
            "year",
            "month",
            "quarter",
            "holiday",
            "month_sin",
            "month_cos",
            "quarter_sin",
            "quarter_cos",
        ]
    ],
    on=["year", "month"],
    how="left",
)

volume_monthly_df_best.isnull().sum()

poc_ppg_id                    0
ppg_name                      0
unique_poc_id                 0
year                          0
month                         0
sales_volume_hl_agg           0
year_month_start_date         0
sample_submission_flag        0
id_life_flag                  0
poc_ppg_cluster               0
category_name                 0
price_segment                 0
brand                         0
subbrand                      0
subchannel                    0
channel_type                  0
segment                       0
ppg_seasonality_index         0
poc_seasonality_index         0
sales_lag_12                  0
sales_lag_13                  0
sales_lag_14                  0
sales_lag_15                  0
sales_lag_13_diff             0
sales_lag_14_diff             0
sales_lag_15_diff             0
sales_roll_mean_3             0
sales_roll_mean_6             0
sales_roll_mean_9             0
sales_ewm_alpha_095_lag_12    0
sales_ewm_alpha_095_lag_13    0
sales_ew

In [87]:
volume_monthly_df_best.shape

(2565882, 46)

In [88]:
volume_monthly_df_best = pd.merge(
    volume_monthly_df_best,
    covid_data_monthly[
        [
            "year",
            "month",
            "daily_cases",
            "daily_deaths",
            "daily_cases_band",
            "daily_deaths_band",
        ]
    ],
    on=["year", "month"],
    how="left",
)

# missing values
volume_monthly_df_best[["daily_cases", "daily_deaths"]] = volume_monthly_df_best[
    ["daily_cases", "daily_deaths"]
].fillna(0.0)

volume_monthly_df_best[["daily_cases_band", "daily_deaths_band"]] = (
    volume_monthly_df_best[["daily_cases_band", "daily_deaths_band"]]
    .astype(str)
    .replace("nan", "unknown")
)

volume_monthly_df_best.isnull().sum()

poc_ppg_id                    0
ppg_name                      0
unique_poc_id                 0
year                          0
month                         0
sales_volume_hl_agg           0
year_month_start_date         0
sample_submission_flag        0
id_life_flag                  0
poc_ppg_cluster               0
category_name                 0
price_segment                 0
brand                         0
subbrand                      0
subchannel                    0
channel_type                  0
segment                       0
ppg_seasonality_index         0
poc_seasonality_index         0
sales_lag_12                  0
sales_lag_13                  0
sales_lag_14                  0
sales_lag_15                  0
sales_lag_13_diff             0
sales_lag_14_diff             0
sales_lag_15_diff             0
sales_roll_mean_3             0
sales_roll_mean_6             0
sales_roll_mean_9             0
sales_ewm_alpha_095_lag_12    0
sales_ewm_alpha_095_lag_13    0
sales_ew

In [89]:
volume_monthly_df_best.shape

(2565882, 50)

In [90]:
# weather details
volume_monthly_df_best = pd.merge(
    volume_monthly_df_best,
    weather_data_monthly[
        [
            "year",
            "month",
            "tempmax",
            "tempmin",
            "temp",
            "dew",
            "humidity",
            "precip",
            "windgust",
        ]
    ],
    on=["year", "month"],
    how="left",
)

In [91]:
volume_monthly_df_best.isnull().sum()

poc_ppg_id                    0
ppg_name                      0
unique_poc_id                 0
year                          0
month                         0
sales_volume_hl_agg           0
year_month_start_date         0
sample_submission_flag        0
id_life_flag                  0
poc_ppg_cluster               0
category_name                 0
price_segment                 0
brand                         0
subbrand                      0
subchannel                    0
channel_type                  0
segment                       0
ppg_seasonality_index         0
poc_seasonality_index         0
sales_lag_12                  0
sales_lag_13                  0
sales_lag_14                  0
sales_lag_15                  0
sales_lag_13_diff             0
sales_lag_14_diff             0
sales_lag_15_diff             0
sales_roll_mean_3             0
sales_roll_mean_6             0
sales_roll_mean_9             0
sales_ewm_alpha_095_lag_12    0
sales_ewm_alpha_095_lag_13    0
sales_ew

In [92]:
volume_monthly_df_best.shape

(2565882, 57)

##### macro

In [93]:
best_kpis = [
    'consumer price index, aop',
    'retail sales, volume index',
    'retail sales, value index',
    'exchange rate lc per usd, aop',
    'production index, industry',
    'inflation, cpi, aop',
    'interest rate, short-term',
    'effective exchange rate index, nominal',
    'retail sales, volume index, % change previous year',
    'wholesale price index',
    "year",
    "month"
]

In [94]:
# weather details
volume_monthly_df_best = pd.merge(
    volume_monthly_df_best,
    macro_df[best_kpis],
    on=["year", "month"],
    how="left",
)

In [95]:
volume_monthly_df_best.head()

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date,sample_submission_flag,id_life_flag,poc_ppg_cluster,category_name,price_segment,brand,subbrand,subchannel,channel_type,segment,ppg_seasonality_index,poc_seasonality_index,sales_lag_12,sales_lag_13,sales_lag_14,sales_lag_15,sales_lag_13_diff,sales_lag_14_diff,sales_lag_15_diff,sales_roll_mean_3,sales_roll_mean_6,sales_roll_mean_9,sales_ewm_alpha_095_lag_12,sales_ewm_alpha_095_lag_13,sales_ewm_alpha_095_lag_14,sales_ewm_alpha_09_lag_12,sales_ewm_alpha_09_lag_13,sales_ewm_alpha_09_lag_14,sales_ewm_alpha_08_lag_12,sales_ewm_alpha_08_lag_13,sales_ewm_alpha_08_lag_14,maco_hl,nr_hl,quarter,holiday,month_sin,month_cos,quarter_sin,quarter_cos,daily_cases,daily_deaths,daily_cases_band,daily_deaths_band,tempmax,tempmin,temp,dew,humidity,precip,windgust,"consumer price index, aop","retail sales, volume index","retail sales, value index","exchange rate lc per usd, aop","production index, industry","inflation, cpi, aop","interest rate, short-term","effective exchange rate index, nominal","retail sales, volume index, % change previous year",wholesale price index
0,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,9,2.657029,2021-09-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.086632,0.083310,2.680361,2.659049,2.641487,2.663112,-0.021312,-0.017561,0.021625,2.675780,2.652467,2.650496,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,63.695448,113.044610,3,1.0,-1.000000e+00,-1.836970e-16,-1.000000e+00,-1.836970e-16,125013.0,5365.0,high,high,18.65,9.5,14.15,8.20,68.40,0.000,43.35,109.067272,99.803771,108.842318,14.566128,93.946973,4.965848,3.79,99.876129,2.243897,111.255241
1,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,10,1.635095,2021-10-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.097920,0.099356,2.660838,2.665773,2.663828,2.645569,0.004934,-0.001944,-0.018259,2.659772,2.658145,2.643428,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,61.805912,111.155075,4,0.0,-8.660254e-01,5.000000e-01,-2.449294e-16,1.000000e+00,19444.0,1551.0,low,mid,22.00,13.0,17.20,10.10,68.00,0.001,44.90,109.544098,100.516543,110.098882,14.847482,89.044522,4.964110,3.75,98.607810,2.394505,112.176646
2,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,11,5.477568,2021-11-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.116346,0.114571,2.663475,2.656727,2.663527,2.641021,-0.006748,0.006800,-0.022506,2.663179,2.658387,2.659077,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,62.990969,112.155744,4,0.0,-5.000000e-01,8.660254e-01,-2.449294e-16,1.000000e+00,45936.0,666.0,mid,mid,22.00,15.0,18.10,11.75,65.45,0.000,46.30,110.214387,100.150272,110.368923,15.480569,93.946973,5.379141,3.89,95.304895,2.178436,114.017164
3,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,12,4.905285,2021-12-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.135510,0.138193,2.645500,2.674356,2.650064,2.654775,0.028856,-0.024293,0.004711,2.663731,2.652931,2.670176,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,63.164604,111.637664,4,4.0,-2.449294e-16,1.000000e+00,-2.449294e-16,1.000000e+00,490234.0,1302.0,high,mid,23.00,15.8,19.30,13.20,64.50,0.000,51.50,111.006049,101.133586,112.253124,15.850507,94.647324,5.915687,3.83,93.725565,2.450774,115.774565
4,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2022,1,0.613161,2022-01-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.056200,0.055705,2.635591,2.657807,2.671511,2.658479,0.022217,0.013704,-0.013033,2.675261,2.660880,2.669944,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,57.546540,106.895703,1,1.0,5.000000e-01,8.660254e-01,1.000000e+00,6.123234e-17,146936.0,3948.0,high,high,27.50,17.7,22.00,15.50,64.70,0.000,41.80,111.499937,102.457158,114.228193,15.492598,96.648324,5.711628,4.03,95.625194,7.804066,116.362157


In [96]:
volume_monthly_df_best.isnull().sum()

poc_ppg_id                                            0
ppg_name                                              0
unique_poc_id                                         0
year                                                  0
month                                                 0
sales_volume_hl_agg                                   0
year_month_start_date                                 0
sample_submission_flag                                0
id_life_flag                                          0
poc_ppg_cluster                                       0
category_name                                         0
price_segment                                         0
brand                                                 0
subbrand                                              0
subchannel                                            0
channel_type                                          0
segment                                               0
ppg_seasonality_index                           

In [97]:
volume_monthly_df_best.shape

(2565882, 67)

In [98]:
volume_monthly_df_best.drop_duplicates(['poc_ppg_id','ppg_name','unique_poc_id','year','month']).shape

(2565882, 67)

###### only create feature for sample submissions

In [99]:
volume_monthly_df_best.to_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "submission_id_vol_cluster_monthly_df_all_feats.pkl"
    )
)

In [100]:
volume_monthly_df_best.head(3)

,poc_ppg_id,ppg_name,unique_poc_id,year,month,sales_volume_hl_agg,year_month_start_date,sample_submission_flag,id_life_flag,poc_ppg_cluster,category_name,price_segment,brand,subbrand,subchannel,channel_type,segment,ppg_seasonality_index,poc_seasonality_index,sales_lag_12,sales_lag_13,sales_lag_14,sales_lag_15,sales_lag_13_diff,sales_lag_14_diff,sales_lag_15_diff,sales_roll_mean_3,sales_roll_mean_6,sales_roll_mean_9,sales_ewm_alpha_095_lag_12,sales_ewm_alpha_095_lag_13,sales_ewm_alpha_095_lag_14,sales_ewm_alpha_09_lag_12,sales_ewm_alpha_09_lag_13,sales_ewm_alpha_09_lag_14,sales_ewm_alpha_08_lag_12,sales_ewm_alpha_08_lag_13,sales_ewm_alpha_08_lag_14,maco_hl,nr_hl,quarter,holiday,month_sin,month_cos,quarter_sin,quarter_cos,daily_cases,daily_deaths,daily_cases_band,daily_deaths_band,tempmax,tempmin,temp,dew,humidity,precip,windgust,"consumer price index, aop","retail sales, volume index","retail sales, value index","exchange rate lc per usd, aop","production index, industry","inflation, cpi, aop","interest rate, short-term","effective exchange rate index, nominal","retail sales, volume index, % change previous year",wholesale price index
0,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,9,2.657029,2021-09-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.086632,0.083310,2.680361,2.659049,2.641487,2.663112,-0.021312,-0.017561,0.021625,2.675780,2.652467,2.650496,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,63.695448,113.044610,3,1.0,-1.000000,-1.836970e-16,-1.000000e+00,-1.836970e-16,125013.0,5365.0,high,high,18.65,9.5,14.15,8.20,68.40,0.000,43.35,109.067272,99.803771,108.842318,14.566128,93.946973,4.965848,3.79,99.876129,2.243897,111.255241
1,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,10,1.635095,2021-10-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.097920,0.099356,2.660838,2.665773,2.663828,2.645569,0.004934,-0.001944,-0.018259,2.659772,2.658145,2.643428,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,61.805912,111.155075,4,0.0,-0.866025,5.000000e-01,-2.449294e-16,1.000000e+00,19444.0,1551.0,low,mid,22.00,13.0,17.20,10.10,68.00,0.001,44.90,109.544098,100.516543,110.098882,14.847482,89.044522,4.964110,3.75,98.607810,2.394505,112.176646
2,AAAC__CJ_C_FN_M,CJ C FN M,AAAC,2021,11,5.477568,2021-11-01,1.0,1.0,1_1,BM,EJ CW,CJ,CJ C,AQXZ APPZ ADOT,BBOB,AZJC UIP,0.116346,0.114571,2.663475,2.656727,2.663527,2.641021,-0.006748,0.006800,-0.022506,2.663179,2.658387,2.659077,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,2.657029,62.990969,112.155744,4,0.0,-0.500000,8.660254e-01,-2.449294e-16,1.000000e+00,45936.0,666.0,mid,mid,22.00,15.0,18.10,11.75,65.45,0.000,46.30,110.214387,100.150272,110.368923,15.480569,93.946973,5.379141,3.89,95.304895,2.178436,114.017164


In [101]:
volume_monthly_df_best.groupby(["poc_ppg_cluster"]).agg({"poc_ppg_id": "nunique"})

,poc_ppg_id
poc_ppg_cluster,
0_0,545
0_1,9442
1_0,5311
1_1,34039


##### We create the ppg month level low and top percentile values(5-99)

In [5]:
volume_monthly_df_best = pd.read_pickle(
    os.path.join(
        os.getcwd(), processed_data_folder, "all_id_vol_cluster_monthly_df.pkl"
    )
)

In [21]:
volume_monthly_df_best.columns

Index(['poc_ppg_id', 'ppg_name', 'unique_poc_id', 'year', 'month',
       'sales_volume_hl_agg', 'year_month_start_date',
       'sample_submission_flag', 'id_life_flag', 'poc_ppg_cluster'],
      dtype='object')

In [25]:
temp = volume_monthly_df_best.groupby(['poc_ppg_id','month'],as_index=False).agg(low_band = ("sales_volume_hl_agg",lambda x: np.percentile(x,5)),
                                                                                high_band = ("sales_volume_hl_agg",lambda x: np.percentile(x,99.5))
                                                                                )

KeyboardInterrupt: 

In [ ]:
temp.head()